In [ ]:
from langchain.tools import tool

@tool
def search_database(query: str, limit: int = 10) -> str:
    """Search the customer database for records matching the query.

    Args:
        query: Search terms to look for
        limit: Maximum number of results to return
    """
    return f"Found {limit} results for '{query}'"

customize tool properties

In [ ]:
@tool("web_search")  # Custom name
def search(query: str) -> str:
    """Search the web for information."""
    return f"Results for: {query}"

print(search.name)  # web_search

Custom tool description


In [ ]:
@tool("calculator", description="Performs arithmetic calculations. Use this for any math problems.")
def calc(expression: str) -> str:
    """Evaluate mathematical expressions."""
    return str(eval(expression))

Advanced schema definition


In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

class WeatherInput(BaseModel):
    """Input for weather queries."""
    location: str = Field(description="City name or coordinates")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="Temperature unit preference"
    )
    include_forecast: bool = Field(
        default=False,
        description="Include 5-day forecast"
    )

@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """Get current weather and optional forecast."""
    temp = 22 if units == "celsius" else 72
    result = f"Current weather in {location}: {temp} degrees {units[0].upper()}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result

Accessing state:


In [ ]:
from typing_extensions import Annotated
from langchain.tools import InjectedState

# Access the current conversation state
@tool
def summarize_conversation(
    state: Annotated[dict, InjectedState]
) -> str:
    """Summarize the conversation so far."""
    messages = state["messages"]

    human_msgs = sum(1 for m in messages if m.__class__.__name__ == "HumanMessage")
    ai_msgs = sum(1 for m in messages if m.__class__.__name__ == "AIMessage")
    tool_msgs = sum(1 for m in messages if m.__class__.__name__ == "ToolMessage")

    return f"Conversation has {human_msgs} user messages, {ai_msgs} AI responses, and {tool_msgs} tool results"

# Access custom state fields
@tool
def get_user_preference(
    pref_name: str,
    preferences: Annotated[dict, InjectedState("user_preferences")]  # InjectedState parameters are not visible to the model
) -> str:
    """Get a user preference value."""
    return preferences.get(pref_name, "Not set")

updating state

In [ ]:
from langgraph.types import Command
from langchain.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langchain.tools import tool, InjectedToolCallId
from typing_extensions import Annotated

# Update the conversation history by removing all messages
@tool
def clear_conversation() -> Command:
    """Clear the conversation history."""

    return Command(
        update={
            "messages": [RemoveMessage(id=REMOVE_ALL_MESSAGES)],
        }
    )

# Update the user_name in the agent state
@tool
def update_user_name(
    new_name: str,
    tool_call_id: Annotated[dict, InjectedToolCallId]
) -> Command:
    """Update the user's name."""
    return Command(update={"user_name": new_name})

Context runtime

In [1]:
from langchain_ollama import ChatOllama
model = ChatOllama(model="gpt-oss:120b-cloud")

In [2]:
from dataclasses import dataclass
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool
from langgraph.runtime import get_runtime

USER_DATABASE = {
    "user123": {
        "name": "Alice Johnson",
        "account_type": "Premium",
        "balance": 5000,
        "email": "alice@example.com"
    },
    "user456": {
        "name": "Bob Smith",
        "account_type": "Standard",
        "balance": 1200,
        "email": "bob@example.com"
    }
}

@dataclass
class UserContext:
    user_id: str

@tool
def get_account_info() -> str:
    """Get the current user's account information."""
    runtime = get_runtime(UserContext)
    user_id = runtime.context.user_id

    if user_id in USER_DATABASE:
        user = USER_DATABASE[user_id]
        return f"Account holder: {user['name']}\nType: {user['account_type']}\nBalance: ${user['balance']}"
    return "User not found"

#model = ChatOpenAI(model="gpt-4o")
agent = create_agent(
    model,
    tools=[get_account_info],
    context_schema=UserContext,
    system_prompt="You are a financial assistant."
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my current balance?"}]},
    context=UserContext(user_id="user123")
)

In [3]:
result

{'messages': [HumanMessage(content="What's my current balance?", additional_kwargs={}, response_metadata={}, id='35600bfb-df91-4aab-960a-b47107b1a0cc'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:120b-cloud', 'created_at': '2025-10-15T07:00:40.087590395Z', 'done': True, 'done_reason': 'stop', 'total_duration': 618268103, 'load_duration': None, 'prompt_eval_count': 117, 'prompt_eval_duration': None, 'eval_count': 43, 'eval_duration': None, 'model_name': 'gpt-oss:120b-cloud', 'model_provider': 'ollama'}, id='lc_run--f453e0e8-059d-4b19-bc99-179b53440247-0', tool_calls=[{'name': 'get_account_info', 'args': {}, 'id': '93c35395-bc31-4e6f-b490-d09a5d81ba1c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 117, 'output_tokens': 43, 'total_tokens': 160}),
  ToolMessage(content='Account holder: Alice Johnson\nType: Premium\nBalance: $5000', name='get_account_info', id='20c9adc1-7a6d-47b0-bdaa-7b10dd968126', tool_call_id='93c35395-bc31-4e6f-b490-d09a

memory

In [4]:
from typing import Any
from langgraph.runtime import get_runtime
from langgraph.store.memory import InMemoryStore
from langchain.agents import create_agent
from langchain.tools import tool

# Access memory
@tool
def get_user_info(user_id: str) -> str:
    """Look up user info."""
    store = get_runtime().store
    user_info = store.get(("users",), user_id)
    return str(user_info.value) if user_info else "Unknown user"

# Update memory
@tool
def save_user_info(user_id: str, user_info: dict[str, Any]) -> str:
    """Save user info."""
    store = get_runtime().store
    store.put(("users",), user_id, user_info)
    return "Successfully saved user info."

store = InMemoryStore()
agent = create_agent(
    model,
    tools=[get_user_info, save_user_info],
    store=store
)

# First session: save user info
agent.invoke({
    "messages": [{"role": "user", "content": "Save the following user: userid: abc123, name: Foo, age: 25, email: foo@langchain.dev"}]
})

# Second session: get user info
agent.invoke({
    "messages": [{"role": "user", "content": "Get user info for user with id 'abc123'"}]
})
# Here is the user info for user with ID "abc123":
# - Name: Foo
# - Age: 25
# - Email: foo@langchain.dev

{'messages': [HumanMessage(content="Get user info for user with id 'abc123'", additional_kwargs={}, response_metadata={}, id='25a2f870-91b1-44bb-9f57-2d7addb0563f'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:120b-cloud', 'created_at': '2025-10-15T07:01:58.889630788Z', 'done': True, 'done_reason': 'stop', 'total_duration': 842255959, 'load_duration': None, 'prompt_eval_count': 161, 'prompt_eval_duration': None, 'eval_count': 47, 'eval_duration': None, 'model_name': 'gpt-oss:120b-cloud', 'model_provider': 'ollama'}, id='lc_run--fef1f9e0-7a8d-424c-80dd-7f264421da13-0', tool_calls=[{'name': 'get_user_info', 'args': {'user_id': 'abc123'}, 'id': '1169c18e-bff4-45bd-ae04-254326b6c1f5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 161, 'output_tokens': 47, 'total_tokens': 208}),
  ToolMessage(content="{'age': 25, 'email': 'foo@langchain.dev', 'name': 'Foo'}", name='get_user_info', id='08b88885-9f0a-4fa3-9f2e-ca644e5d5579', tool_call_id='1169c1

Stream Writer


In [ ]:
from langchain.tools import tool
from langgraph.runtime import get_runtime

@tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_runtime().stream_writer

    # Stream custom updates as the tool executes
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")

    return f"It's always sunny in {city}!"